In [1]:
print("This code was designed to work with the following packages :\nfeedparser 5.1.3\npandas 0.18.1\nnumpy 1.11.1\nfolium 0.7.0\nmatplotlib 1.5.1\nbranca 0.3.1\njinja2 2.8\ndatetime ")


import feedparser #5.1.3
import pandas as pd #0.18.1
import numpy as np #1.11.1
import folium #0.7.0
import matplotlib #1.5.1

from folium import branca #0.3.1
from folium.plugins import MarkerCluster
import folium.plugins 

import tkinter as tk #8.6

from branca.element import MacroElement #0.3.1

from jinja2 import Template  #2.8

from datetime import datetime, timedelta
import dateutil.relativedelta

import tkinter as tk


This code was designed to work with the following packages :
feedparser 5.1.3
pandas 0.18.1
numpy 1.11.1
folium 0.7.0
matplotlib 1.5.1
branca 0.3.1
jinja2 2.8
datetime 


In [2]:
x = pd.read_csv("/home/gfa/Geospatial data/Project/areas.csv", delimiter = ';',  encoding='latin-1')
#REM ! had to change the "encoding" method => more explication can be found here :
#https://stackoverflow.com/questions/5552555/unicodedecodeerror-invalid-continuation-byte  07/12/2018

area = input('Give a region : ')
matching = [s for s in x['Region'] if area in s]

selected = []
def select():
    global selected
    selected.append(var.get())
    root.title(selected[-1])
    
root = tk.Tk()
# use width x height + x_offset + y_offset (no spaces!)
root.geometry("%dx%d+%d+%d" % (400, 50, 200, 150))
root.title("tk.Optionmenu as combobox")
var = tk.StringVar(root)

# initial value
var.set(matching[0])
choices = matching
option = tk.OptionMenu(root, var, *choices)
option.pack(side='left', padx=10, pady=10)
button = tk.Button(root, text="check value selected", command=select)
button.pack(side='left', padx=20, pady=10)

root.mainloop()

Give a region : Alaska


In [3]:
j=0
RSSentry =[]
for q in selected :
    i=0;
    for p in x['Region'] :

        if selected[j] == p : 
            index = i;
        else :
            i+=1;       
    j+=1;
    RSSentry.append(x['RSS'][index])
i=0
for p in RSSentry :
    RSSentry[i]=RSSentry[i].upper()
    RSSentry[i]=RSSentry[i].replace(" ","+")
    i+=1;
RSSentry
#CHANGE TO CAPITAL LETTERS + REPLACE ' ' by a + 

['CENTRAL+ALASKA', 'SOUTH+OF+ALASKA', 'NORTHERN+ALASKA']

In [4]:
#Problem if region have no known event => check entries size => bypass if <1
df=pd.DataFrame(columns=['Longitude', 'Latitude', 'Magnitude'])
for p in RSSentry :
    area = p
    
    rss_url = "https://www.emsc-csem.org/service/rss/rss.php?filter=yes&region={}&min_intens=0&max_intens=8".format(area)

    feed = feedparser.parse(rss_url)   #Recovery of RSS data
    if len(feed.entries)== 1 : 
        print("No events")
    
    else :
        entry= feed.entries[1]

        x=[[i.geo_long, i.geo_lat, i.emsc_magnitude, i.emsc_depth, i.emsc_time] for i in feed.entries]
        #Double list to extract data
        df2=pd.DataFrame(x, columns=['Longitude', 'Latitude', 'Magn', 'Prof', 'date'])
        #Creating a dataframe with columns(names)
        df2['Type_magnitude'], df2['Magnitude'] = df2['Magn'].str.split(' ', 1).str
        df2['Depth'], df2['Type_depth'] = df2['Prof'].str.split(' ', 1).str
        #df2['Date'], df2['Heure'] = df2['date'].str.split(' ', 1).str
        #Columns separation to optimize the dataframe 
        del df2['Magn']
        del df2['Prof']
        #Erase unnecessary columns  
        df = df.append(df2, ignore_index = True)
        

i=0
for p in df['Magnitude']:
    df['Magnitude'][i]= float(df['Magnitude'][i])
    df['Longitude'][i]= float(df['Longitude'][i])
    df['Latitude'][i]= float(df['Latitude'][i])
    df['Depth'][i]= float(df['Depth'][i])
    i+=1
#Transformation of the value into a float

In [5]:
i=0
for p in df['date']:
    df['date'][i]=df['date'][i].replace(' UTC','')
    df['date'][i]=datetime.strptime( df['date'][i],'%Y-%m-%d %H:%M:%S')
    i+=1

In [7]:
#Defining the color in function of date
now=datetime.now()
def colortest(date):
    yesterday = datetime.now() - timedelta(days = 1)
    week=  datetime.now() - timedelta(days = 7)
    month=  now + dateutil.relativedelta.relativedelta(months=-1)
    year=   now + dateutil.relativedelta.relativedelta(years=-1)
    timetest=now-date
    if (date<year):
        col='gray'
    elif (date>year and date<month):
        col='green'
    elif (date>month and date<week):
        col='yellow'
    elif (date>week and date<yesterday):
        col='blue'
    else :
        col='red'
    return col

In [11]:
#Defining the colormaps we will use for each event
linear_magn = branca.colormap.LinearColormap(
    ['green', 'yellow','orange', 'red','blue'],
    vmin=min(df['Magnitude']), vmax=max(df['Magnitude'])
)
linear_magn.caption = 'Magnitude'

linear_prof = branca.colormap.LinearColormap(
    ['green', 'yellow','orange', 'red','blue'],
    vmin=min(df['Depth']), vmax=max(df['Depth'])
)
linear_prof.caption = 'Depth'

#Magnitude part
cmagn=folium.FeatureGroup(name='Magnitude')   #Creation of a new layer
m = folium.Map(control_scale=True)            #Creation of a map
marker_cluster = MarkerCluster()              #Creation of Marker Cluster
for i in range(0,len(df['Magnitude']),1):
    markers= folium.CircleMarker([df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
                  popup=str(df.iloc[i]['Magnitude']),
                  radius=df.iloc[i]['Magnitude']*5,
                  color=linear_magn(df.iloc[i]['Magnitude']),
                  fill_color=linear_magn(df.iloc[i]['Magnitude']))   #Creation of Marker with color depending of the Magnitude value
    marker_cluster.add_child(markers)
    
cmagn.add_child(marker_cluster)

#Depth part

cprof=folium.FeatureGroup(name='Depth')
marker_cluster2 = MarkerCluster()
for i in range(0,len(df['Depth']),1):
    markers2= folium.CircleMarker([df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
                  popup=str(df.iloc[i]['Depth']),
                  radius=df.iloc[i]['Depth']*2,
                  color=linear_prof(df.iloc[i]['Depth']),
                  fill_color=linear_prof(df.iloc[i]['Depth']))
    marker_cluster2.add_child(markers2)

cprof.add_child(marker_cluster2)

#Date part

cdate=folium.FeatureGroup(name='date')
marker_cluster3 = MarkerCluster()
for i in range(0,len(df['date']),1):
    markers3= folium.CircleMarker([df.iloc[i]['Latitude'], df.iloc[i]['Longitude']],
                  popup=str(df.iloc[i]['date']),
                  radius=df.iloc[i]['Magnitude']*5,
                  color=colortest(df.iloc[i]['date']),
                  fill_color=colortest(df.iloc[i]['date']))
    marker_cluster3.add_child(markers3)

cdate.add_child(marker_cluster3)


liste_points=[]
for i in range(0,len(df['Magnitude']),1):
    liste_points.append([df['Latitude'][i],df['Longitude'][i]])
    i+=1
xx=[liste_points]
m.fit_bounds(xx)

#Part of code that I copy/paste, still have to figure out how it works
#Source : http://nbviewer.jupyter.org/gist/BibMartin/f153aa957ddc5fadc64929abdee9ff2e 07/12/2018

from branca.element import MacroElement

from jinja2 import Template

class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)  # noqa
        
#Adding all the layers to the map
m.add_child(cmagn).add_child(cprof).add_child(cdate)
m.add_child(folium.map.LayerControl())
m.add_child(linear_magn).add_child(linear_prof)
#Binding colormaps and layers using the "BlindColormap" function defined above
                             
m.add_child(BindColormap(cmagn,linear_magn)).add_child(BindColormap(cprof, linear_prof))
m

outf = "/home/gfa/Geospatial data/Project/map.html"
m.save(outf)
import webbrowser

webbrowser.open(outf) 

True